In [2]:
# Import libraries
import numpy as np
import geopandas as gpd
import momepy
import time
import copy
import networkx as nx
# import pandas as pd
# import shapely
# import shapely.geometry as sg
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline

from lmzintgraf_gp_pref_elicit import dataset, gaussian_process, acquisition_function
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_ccs as utils_ccs
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_data as utils_data
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_experiment as utils_experiment
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_parameters as utils_parameters
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_user as utils_user

In [8]:
map = gpd.read_file("Sidewalk_width_crossings_smaller.geojson") #Read in the map with radius 250m and ~1000 nodes

# Objectives
objective1 = map['length']
objective2 = map['crossing']
objective3 = map['obstacle_free_width']

objectives = ('length', 'crossing')

In [9]:
# Create a NetworkX graph from the map
G = momepy.gdf_to_nx(map, approach='primal')
nodes = G.nodes
edges = G.edges

/usr/local/anaconda3/envs/test/lib/python3.10/site-packages/momepy/utils.py:252: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


In [31]:
# print(nodes)

In [14]:
#Pick random ones or pick manually that make sense - to experiment
S = (122245.37633330293, 486126.8581684635)
T = (122246.77932030056, 486223.5791244763)

In [32]:
def pareto_dominates(a, b):
    """Check if the vector in b Pareto dominates vector a.

    Note: The original code has been modified to work for our minimization problem.

    Args:
        a (ndarray): A numpy array.
        b (ndarray): A numpy array.

    Returns:
        bool: Whether vector b dominates vector a.
    """
    a = np.array(a)
    b = np.array(b)
    return np.all(a <= b) and np.any(a < b)


def p_prune(candidates):
    """Create a Pareto coverage set from a set of candidate points.

    References:
        .. [1] Roijers, D. M., & Whiteson, S. (2017). Multi-objective decision making. 34, 129–129.
            https://doi.org/10.2200/S00765ED1V01Y201704AIM034

    Args:
        candidates (Set[Tuple]): A set of vectors.

    Returns:
        Set[Tuple]: A Pareto coverage set.
    """
    pcs = set()
    while candidates:
        vector = candidates.pop()

        for alternative in candidates:
            if pareto_dominates(alternative, vector):
                vector = alternative

        to_remove = set(vector)
        for alternative in candidates:
            if pareto_dominates(vector, alternative):
                to_remove.add(alternative)

        candidates -= to_remove
        pcs.add(vector)
    return pcs


def pvi(G, init_pcs, objectives, max_iter=10, epsilon=0.05):
    # num_states = len(G.nodes)
    start = time.time()
    cost = []
    nd_vectors = init_pcs
    nd_vectors_update = copy.deepcopy(nd_vectors)

    last_iter = max_iter - 2  # Denotes the start of the last run.
    is_last = False
    save_iteration = False

    for run in range(max_iter):  # We execute the algorithm for a number of iterations.
        print(f'Value Iteration number: {run}')
        for n, current_node in enumerate(G.nodes):  # Loop over all states. Note: current_node is an object; n=number
            # print(f'Looping over state {n}')
            for nk, neighbor in enumerate(G.neighbors(current_node)): #Note: neighbor is an object; k=number
                edge = G[current_node][neighbor]
                edge_list = [v for k, v in edge.items()]  # Stores only the values of the edges' properties
                for i in objectives:
                  cost = np.array([edge_list[0][i]])

                results = {tuple(cost + value_vec) for value_vec in nd_vectors[n][nk]}  # The set of candidate vectors.
                results = p_prune(results)
                nd_vectors_update[n][nk] = results
        if is_last:
            break
        nd_vectors = copy.deepcopy(nd_vectors_update)  # Else perform a deep copy an go again.

    end = time.time()
    elapsed_seconds = (end - start)
    print("Seconds elapsed: " + str(elapsed_seconds))

    return nd_vectors_update


# for each action in A:
# retireve R=cost
# Q is set of value vector
# s' is neighbor
# T is 1
#actions = neighbors

def track_policy(G, t, objectives, pvi_vec):
    for n, current_node in enumerate(G.nodes):  # Loop over all states. Note: current_node is an object; n=number
        if n == T:
            break

        for nk, neighbor in enumerate(G.neighbors(current_node)):
            edge = G[current_node][neighbor]
            edge_list = [v for k, v in edge.items()]  # Stores only the values of the edges' properties
            for i in objectives:
                cost = np.array([edge_list[0][i]])

            for val_vec in pvi_vec:
                print(val_vec)
                if (val_vec + cost) == t:
                    current_node = neighbor
                    t = val_vec


# def track_policy(init_pcs, G, target, tol=1e-3):
#         """Track a policy from its return vector.
#
#         Args:
#             vec (array_like): The return vector to track.
#             G: Graph
#             tol (float, optional): The tolerance for the return vector. (Default value = 1e-3)
#         """
#         # target = np.array(vec)
#         terminated = False
#         truncated = False
#         total_rew = np.zeros(2)
#
#         while not (terminated or truncated):
#             # state = np.ravel_multi_index(state, self.env_shape)
#             closest_dist = np.inf
#             closest_action = 0
#             found_action = False
#             new_target = target
#
#             for n, current_node in enumerate(G.nodes):
#                 for nk, neighbor in enumerate(G.neighbors(current_node)):
#                     im_rew = np.zeros((n, nk, 2))[n, nk]
#                     non_dominated_set = init_pcs[n][nk]
#
#                     for q in non_dominated_set:
#                         q = np.array(q)
#                         dist = np.sum(np.abs(q + im_rew - t))
#                         if dist < closest_dist:
#                             closest_dist = dist
#                             closest_action = nk
#                             new_target = q
#
#                             if dist < tol:
#                                 found_action = True
#                                 break
#
#                     if found_action:
#                         break
#
#                 #closest action = edge you want to take
#                 #step = go to neighbor node
#                 state, reward, terminated, truncated, _ = G.nodes(closest_action) #go to the node that it's at the end of the edge
#                 total_rew += reward
#                 target = new_target
#
#         return total_rew


init_pcs = [[{tuple(np.full(2, [227.59000000000003, 2.0]))} for _ in range(len(G.nodes))] for _ in range(len(G.nodes))]  # Q-set
pvi_result = pvi(G, init_pcs, objectives)
print(pvi_result)
t = np.array([165.21, 2.])
path_pvi = track_policy(G, t, objectives, pvi_result)
# print(path_pvi)

Value Iteration number: 0
Value Iteration number: 1
Value Iteration number: 2
Value Iteration number: 3
Value Iteration number: 4
Value Iteration number: 5
Value Iteration number: 6
Value Iteration number: 7
Value Iteration number: 8
Value Iteration number: 9
Seconds elapsed: 5.685685873031616
[[{(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.59000000000003, 2.0)}, {(227.5

TypeError: unsupported operand type(s) for +: 'set' and 'int'

In [ ]:
# def arg_p_prune(candidates, remove_duplicates=True):
#     """A batched and fast version of the Pareto coverage set algorithm.
#
#         Args:
#             candidates (ndarray): A numpy array of vectors.
#             remove_duplicates (bool, optional): Whether to remove duplicate vectors. Defaults to True.
#
#         Returns:
#             ndarray: A Pareto coverage set.
#         """
#     if len(candidates) == 1:
#         return candidates
#
#     uniques, indcs, invs, counts = np.unique(candidates, return_index=True, return_inverse=True, return_counts=True,
#                                              axis=0)
#
#     res_eq = np.all(candidates[:, None, None] <= candidates, axis=-1).squeeze()
#     res_g = np.all(candidates[:, None, None] < candidates, axis=-1).squeeze()
#     c1 = np.sum(res_eq, axis=-1) == counts[invs]
#     c2 = np.any(~res_g, axis=-1)
#     if remove_duplicates:
#         to_keep = np.zeros(len(candidates), dtype=bool)
#         to_keep[indcs] = 1
#     else:
#         to_keep = np.ones(len(candidates), dtype=bool)
#
#     return np.logical_and(c1, c2) & to_keep
#
#
# def fast_p_prune(candidates, remove_duplicates=True):
#     """A batched and fast version of the Pareto coverage set algorithm.
#
#     Args:
#         candidates (ndarray): A numpy array of vectors.
#         remove_duplicates (bool, optional): Whether to remove duplicate vectors. Defaults to True.
#
#     Returns:
#         ndarray: A Pareto coverage set.
#     """
#     return candidates[arg_p_prune(candidates, remove_duplicates=remove_duplicates)]
#
#
#
# v_n = {}  # Value vector
# objectives = ('length', 'crossing')
# cost = 0
# value_vecs = [{np.inf} for n in G]
#
# for n in G:
#     # v_n[n] = np.inf  # Initialisation of nodes
#
#     if n == T:
#         v_n[n] = 0
#
#     result = []
#
#     for neighbor in G.neighbors(n):
#         edge = G[n][neighbor]
#         edge_list = [v for k, v in edge.items()]  # Stores only the values of the edges' properties
#         value_vecs.append([edge_list[0]['length'], edge_list[0]['crossing']])
#         print(value_vecs)
#         for i in objectives:
#             cost = edge_list[0][i]
#
#         result.append(-(cost + value_vecs))
#     pruned = fast_p_prune([result])
#
#     value_vecs[n] = pruned
#
# print(value_vecs)
#
# #Without prune: 177 v_n
# #With prune: 152 v_n


In [ ]:
# def track_policy():
#         """Track a policy from its return vector.
#
#         Args:
#             vec (array_like): The return vector to track.
#             G: Graph
#             tol (float, optional): The tolerance for the return vector. (Default value = 1e-3)
#         """
#         # target = np.array(vec)
#         # state, _ = env.reset()
#         terminated = False
#         truncated = False
#         total_rew = np.zeros(self.num_objectives)
#
#         while not (terminated or truncated):
#             # state = np.ravel_multi_index(state, self.env_shape)
#             closest_dist = np.inf
#             closest_action = 0
#             found_action = False
#             new_target = target
#
#             for n, current_node in enumerate(G.nodes):
#                 for nk, neighbor in enumerate(G.neighbors(current_node)):
#                     im_rew = self.avg_reward[n, nk]
#                     non_dominated_set = self.non_dominated[n][nk]
#
#                     for q in non_dominated_set:
#                         q = np.array(q)
#                         dist = np.sum(np.abs(q + im_rew - t))
#                         if dist < closest_dist:
#                             closest_dist = dist
#                             closest_action = nk
#                             new_target = q
#
#                             if dist < tol:
#                                 found_action = True
#                                 break
#
#                     if found_action:
#                         break
#
#                 #closest action = edge you want to take
#                 #step = go to neighbor node
#                 state, reward, terminated, truncated, _ = G.nodes(closest_action) #go to the node that it's at the end of the edge
#                 total_rew += reward
#                 target = new_target
#
#         return total_rew
